# Data Report

This data report contains information on the data sources used for this project, the data pipeline for cleaning and extracting them and limitations of the chosen data.

The main question for this project to answer is:

**How significant is the impact of the EU Emissions Trading System (EU ETS) in the sectors it sanctions and to what degree does that make an impact on global emissions.**

## Data Sources

<!-- Describe your data sources: Why you have chosen them, where they are from, and what data they contain - What is the data structure and quality of your sources? -->
The data sources chosen to answer the main question of this project with, are data of the EU Emissions Trading System in form of the **EU Transaction Log** (EUTL) to find the exact CO2e emissions per sector, the **development of the CO2e price in the EU** to link that to rising/falling CO2e emissions and finally a dataset that contains **global GHG emissions data** to compare the EU with what's happening on a global scale.

Following table contains a quick overview on the data sources, their origin, license, structure and quality. The quality column references the quality dimensions abbreviated in the order "Accuracy", "Completeness", "Consistency", "Timeliness" and "Relevancy". A "_" means that the quality dimension is not met. 

<!-- Describe the licenses of your data sources, why you are allowed to use the data and how you are planning to follow their obligations
If your source data is under a standard open-data license just pointing out where to find that is enough information for being allowed to use it, please still describe how you plan to fulfill their obligations -->

| Data Source                  | Origin          | License         | Structure  | Quality |
|------------------------------|-----------------|-----------------|------------|---------|
| EU Transaction Log     | The EU          | [CC BY 4.0 DEED](https://creativecommons.org/licenses/by/4.0/)  | Structured | AC_TR   |
| Operators in EUTL            | The EU          | [CC BY 4.0 DEED](https://creativecommons.org/licenses/by/4.0/)  | Structured | ACCTR   |
| CO2e Price Development in EU | Umweltbundesamt | [§ 12a EGovG](https://www.gesetze-im-internet.de/egovg/__12a.html)     | Structured | ACCTR   |
| Global GHG Emissions         | The EU / EDGAR  | [CC BY-NC-ND 4.0](https://creativecommons.org/licenses/by-nc-nd/4.0) | Structured | ACCTR   |

The two datasets concerning the EUTL are both licensed under the CC BY 4.0 DEED license, which means the data can be adapted and shared as long as appropriate credit and a link to the license is given. The dataset from the Umweltbundesamt is published under § 12a EGovG, which allows any citizen to freely access and process the data. Additionally when using the data, the Umweltbundesamt has to be mentioned. The global GHG emissions dataset is licensed under the CC BY-NC-ND 4.0 license, which allows to format and redistribute the data as long as the creator is mentioned.

<!-- Include figures and tables to give an overview of data pipeline structure or data sources -->

<!-- Include figures and tables to give an overview of data pipeline structure or data sources -->

## Data Pipeline
<!-- Describe your data pipeline on a high level, which technology did you use to implement it -->
The data pipeline for cleaning the datasets is written in Python using the pandas library. It follows the same pattern for all different data sources that are (1) downloaded from their HTTP resource, (2) cleaned and (3) stored into a SQLite database.

### Cleaning and Transformation
<!-- Which transformation or cleaning steps did you do and why? -->
As all the datasets were already in a structured state only minor cleaning and transformation steps had to be done. Concerning the EUTL the **two sheets of its .xlsx file were joined** into one to improve readability, because the activity types (i.e. sectors) of the companies were stored separate to the main log and were only referenced via an id.

Also the emissions data of the EUTL was split into VERIFIED_EMISSIONS and CH_VERIFIED_EMISSIONS for emissions that stem from the inclusion of Switzerland in the EU ETS. To have a single emission value to work on, these **two columns had to be added up**. Because some of the values in the emission columns were not integers, but resembled another information in string format those were **extracted to a new column**.

<!-- What problems did you encounter and how did you solve them? -->
In the original EUTL dataset, each year was represented by multiple columns that featured e.g. the allocated and verified emissions. To be able to work with the data, I created columns that contained said data for all years by **concatenating each of the previous columns** and **adding a year column** (see table below). I also **removed columns** that I didn't count as important for answering my question.

| Before: | Country | ... | Em_2023   | Alloc_2023 | CH_Em_2023 | ... | Em_2008 | Alloc_2008 |
|---------|---------|-----|-----------|------------|------------|-----|---------|------------|
| After:  | Country | ... | Emissions | Allocation | ...        | ... | ...     | Year       |

Both the EUTL and the dataset on CO2e prices had a **header that had to be removed**. For the CO2e price dataset, **column names had to be manually added** to the dataframe.

### Dealing With Errors
<!-- Describe how your pipeline deals with errors or changing input data -->
The data pipeline does not explicitly account for any changing input data. Nevertheless as all datasets are structured and resemble some kind of log, the pipeline should still be able to process updated versions of these datasets as long as their fundamental structure doesn't change. As for errors, none were apparent in the datasets up until now. Errors that might occur during the execution of the pipeline (i.e. network or OS related) are negligible as they are dependent on the system its running on.

## Results and Limitations

<!-- Describe the output data of your data pipeline - What is the data structure and quality of your result? (Compare lecture D01) -->
The output of the data pipeline is a SQLite database, which contains multiple tables, namely "eu_transaction_log", "eu_ets_operators", "co2e_price_development", "global_ghg_emissions". All the tables contain structured data that aligns with the data quality dimensions mentioned in the lecture.

<!-- What data format did you choose as the output of your pipeline and why -->
SQLite was chosen to store the cleaned data, as the data was and still is structured with fixed data types and it also allows for loading the data into a database viewer if needed.

<!-- Critically reflect on your data and any potential issues you anticipate for your final report -->
**Thoughts on Correctness:** The datasets were mostly published by either the EU or the german state. As the data origins are all governmental and not motivated by e.g. company marketing I reckon the data to be accurate. The EUTL in specific is automatically generated by logging all transactions made between accounts in the EU ETS and ensures that these transactions comply with the EU ETS rules. EDGAR on the other hand, which publishes the dataset on global GHG emissions, also claims to provide independent emission estimates through data collected from EU member states or UNFCCC parties.

**Thoughs on Completeness:** The same governmental argument from before also goes for the completeness of the datasets. The datasets also didn't seem to be missing any crucial information.

**Thoughs on Representativeness:** All datasets grew over the span of decades as they mostly represent the history of CO2e emissions and their pricing over the years up to 2023, which I count as up-to-date. I also reckon no important data was lost or distorted during the cleaning process as no new data was added and any data that was removed was double checked.

Concluding the data report I have a good feeling for the data analysis and final report. So far the data seems viable and not missing any critical information. 